# Process CCP simulation results - Japan Microgrid

In [ ]:
run_from_collab = False

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform

%matplotlib inline

In [ ]:
# Import custom python file from github repo: https://changhsinlee.com/colab-import-python/
if run_from_collab:
    !pip install requests
    import requests
    # Save python as file to colab working directory
    # If you are using GitHub, make sure you get the "Raw" version of the code
    url = 'https://raw.githubusercontent.com/levorato/ccp_rtcs/master/notebooks/rccp_utils.py'
    r = requests.get(url)
    # make sure your filename is the same as how you want to import 
    with open('rccp_utils.py', 'w') as f:
        f.write(r.text)
    # now we can import
    from rccp_utils import *
else:
    from rccp_utils import *

## 1. Process result files

### 1.1. Setup project folders

In [ ]:
if run_from_collab:
    from google.colab import drive
    drive.mount('/content/gdrive/')
    gdrive_folder = '/content/gdrive/MyDrive'
else:
    gdrive_folder = '../..'
print('gdrive_folder=', gdrive_folder)

In [ ]:
project_folder = '../../doutorado/robusto/RCCP'
antoine_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(gdrive_folder, "rccp_experiments")
results_folder = os.path.join(gdrive_folder, "rccp_results")
reportfolder = results_folder
cost_results_folder = os.path.join(output_folder, 'consolidated_results', 'df')
var_results_folder = os.path.join(output_folder, 'consolidated_results', 'df')
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)


In [ ]:
reportfolder = os.path.join(output_folder, 'consolidated_results')
reportfolder_graph = os.path.join(reportfolder, 'graphs')
reportfolder_table = os.path.join(reportfolder, 'tables')
if not os.path.exists(reportfolder_graph):
    os.makedirs(reportfolder_graph)
if not os.path.exists(reportfolder_table):
    os.makedirs(reportfolder_table)
print('Saving files on folder: ' + reportfolder)

### 1.2. List which experiments to process

In [ ]:
experiment_list = ["run_sim_japan_forecast_avg"]

In [ ]:
experiment_folder_list = [os.path.join(output_folder, exp) for exp in experiment_list]
experiment_folder_list

### 1.3. List which CPP models to process

In [ ]:
simulated_model_list = ["robust-budget", "robust-box", "robust-budget"]

### 1.4. Select instance_group to process

In [ ]:
instance_group_list = ["japan-10"]

### 1.5. Select RTCS forecast types to process

In [ ]:
forecast_type_list = ["average"]  # average-based RTCS forecast

In [ ]:
#instances_to_process = ["A_instance2_1000s_skewed-left.txt", "A_instance2_1000s_skewed-right.txt", "A_instance2_1000s_uniform.txt"]
instance_group = "japan-10"
instances_to_process = get_instance_list(project_folder, antoine_instances_folder, toy_instances_folder, japan_instances_folder, instance_group)
instances_to_process

# Optional Steps

### 1.6. Read consolidated result file with model solution / costs

In [ ]:
df_list = []
cost_results_folder = os.path.join(output_folder, 'consolidated_results', 'df')
print(cost_results_folder)
for filepath in glob.glob(os.path.join(cost_results_folder, experiment_list[0] + '.*.cost-results.pkl.gz')):
    df_ = pd.read_pickle(filepath)
    df_list.append(df_)
    if 'deterministic' in filepath:
        print('Read ', filepath)
        print(df_['GammaPerc'].unique())
df_cost = pd.concat(df_list)
del df_list

for file in glob.glob(os.path.join(cost_results_folder, '*deterministic*.cost-results.pkl.gz')):
    result_path = os.path.join(experiment_folder_list[0], file)
    with gzip.open(file,'r') as fh: 
    #with open(file, "rb") as fh:
        df_ = pickle.load(fh)
        #df_ = pd.read_pickle(data)
        #df_['InstanceName'] = 'instance_deltamin10_' + file[file.rfind('_')+1:file.rfind('.csv.gz')]
        df_cost = pd.concat([df_cost, df_])
display(df_cost.info())
display(df_cost['InstanceName'].unique())

### Get the number of scenarios for each instance and model parameter group

In [ ]:
df_check = df_cost[(df_cost['t'] == 1) & (df_cost['d'] == 1)].groupby(by=['InstanceName', 'Model', 'Strategy', 'Reoptimize', 'ModelPolicy', 'ForecastType', 'GammaPerc']).count()
df_check.to_csv(os.path.join(reportfolder, 'japan-check.csv'))


In [ ]:
df_cost.isna().sum()

In [ ]:
df_cost.to_pickle(os.path.join(cost_results_folder, 
                               'run_sim_japan_forecast_avg.japan-10.instance_deltamin10-cost-results-all.pkl.gz'))

### 1.7. Read consolidated result files with model solution / variable values

In [ ]:
df_list = []
for filepath in glob.glob(os.path.join(var_results_folder, experiment_list[0] + '.*.var-results.pkl.gz')):
    with gzip.open(filepath,'r') as fh: 
        #df_ = pickle.load(fh)
        df_ = pd.read_pickle(filepath)
        df_list.append(df_)
        print('Read ', filepath)
df = pd.concat(df_list)
del df_list

In [ ]:
df.head()

In [ ]:
# Convert g_td and h_td from array to scalar, since we heave a single battery
df['g_td'] = [x[0] for x in df['g_td']]  # df['g_td'].astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False).astype(float)
df['h_td'] = [x[0] for x in df['h_td']]  # df['h_td'].astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False).astype(float)

In [ ]:
df.dtypes

In [ ]:
df[['g_td', 'h_td']].describe()

In [ ]:
df.to_pickle(os.path.join(cost_results_folder, 
                               'run_sim_japan_forecast_avg.japan-10.instance_deltamin10-var-results-all.pkl.gz'))

In [ ]:
df.drop(columns=['q_td', 'r_td'], inplace=True)

### 1.8. Merge dataframes

In [ ]:
df = df.merge(df_cost, on=['Model', 'GammaPerc', 'Gamma', 'Strategy', 'Reoptimize', 'ModelPolicy', 'ForecastType', 'ScenarioId', 't', 'd', 'InstanceName'])

In [ ]:
df.info()

### 1.9. Replace the InstanceName column with the season name of each instance

In [ ]:
df['InstanceName'] = df['InstanceName'].str[len('instance_deltamin10_'):-4]
df['GammaPerc'] = df['GammaPerc'].astype(int)
df.head()

### 1.10. Save final merged dataframe to pickle

In [ ]:
df.to_pickle(os.path.join(cost_results_folder, 
                               'run_sim_japan_forecast_avg.japan-10.instance_deltamin10-all-results.pkl.gz'))

In [ ]:
del df_cost

In [ ]:
df.info()

## 2. Final pre-process of dataframe

### 2.0 Load dataframe from disk (skips Step 1 above)

In [ ]:
df = pd.read_pickle(os.path.join(cost_results_folder, 
                               'run_sim_japan_forecast_avg.japan-10.instance_deltamin10-all-results.pkl.gz'))

In [ ]:
df.info()

### 2.1. Create the output folders for processed results

In [ ]:
reportfolder = os.path.join(output_folder, 'consolidated_results')
reportfolder_graph = os.path.join(reportfolder, 'graphs')
reportfolder_table = os.path.join(reportfolder, 'tables')
if not os.path.exists(reportfolder_graph):
    os.makedirs(reportfolder_graph)
if not os.path.exists(reportfolder_table):
    os.makedirs(reportfolder_table)
print('Saving files on folder: ' + reportfolder)

### 2.2. Obtain list of Model, Strategy, ModelPolicy, ForecastType

In [ ]:
model_list = df['Model'].unique().tolist()
strategy_list = df['Strategy'].unique().tolist()
model_policy_list = df['ModelPolicy'].unique().tolist()
reoptimize_value_list = df['Reoptimize'].unique().tolist()
forecast_type_list = df['ForecastType'].unique().tolist()
instances_to_process = df['InstanceName'].unique().tolist()
print("Model", model_list)
print("Strategy", strategy_list)
print("ModelPolicy", model_policy_list)
print("Reoptimize", reoptimize_value_list)
print("ForecastType", forecast_type_list)
print("InstanceName", instances_to_process)

### 2.3. Create 2 new columns: one called ModelName one with the RTCS Policy

* `ModelName` contains MILP model name including parameters (in the budget case)

* `RTCS_Policy` concatenates the info about policy (conservative, audacious, cheapest), look-ahead (i.e., full_model, ignore_model) and model reoptimization (true, false).

In [ ]:
# ModelName
df['ModelName'] = df['Model']
df.loc[(df['Model'] == 'robust-budget'), 'ModelName'] = df.loc[(df['Model'] == 'robust-budget'), 'Model'] + '-'\
    + df.loc[(df['Model'] == 'robust-budget'), 'GammaPerc'].astype(str)
df.loc[(df['Model'] == 'deterministic'), 'ModelName'] = df.loc[(df['Model'] == 'deterministic'), 'Model'] + '-'\
    + df.loc[(df['Model'] == 'deterministic'), 'GammaPerc'].astype(str)
# RTCSPolicy
df['ModelPolicy_temp'] = df['ModelPolicy']
df.loc[(df['ModelPolicy'] == 'ignore_model'), 'ModelPolicy_temp'] = ''
df.loc[(df['ModelPolicy'] == 'full_model'), 'ModelPolicy_temp'] = '+LA'
df['Reoptimize_temp'] = df['Reoptimize'].astype(str)
#df.loc[((df['Reoptimize'] == True) & (df['ModelPolicy'] == 'ignore_model')), 'Reoptimize_temp'] = '+ReOpt'  # '+ReOpt'
df.loc[((df['Reoptimize'] == False) & (df['ModelPolicy'] == 'ignore_model')), 'Reoptimize_temp'] = ''  # '+ReOpt'
df.loc[((df['Reoptimize'] == True) & (df['ModelPolicy'] == 'full_model')), 'Reoptimize_temp'] = '+ReOpt'  # '+ReOpt'
df.loc[((df['Reoptimize'] == False) & (df['ModelPolicy'] == 'full_model')), 'Reoptimize_temp'] = ''  # '+ReOpt'

df['RTCS_Policy'] = df['Strategy'] + df['ModelPolicy_temp'] + df['Reoptimize_temp']
df.drop(columns=['ModelPolicy_temp', 'Reoptimize_temp'], inplace=True)
#df.drop(columns=['Strategy', 'ModelPolicy', 'Reoptimize', 'ForecastType'], inplace=True)
df.head()

In [ ]:
df['RTCS_Policy'].unique().tolist()

### 2.4. Get results only for Lookahead models

## 3. VaR and CVaR functions

In [ ]:
def value_at_risk(returns, confidence_level=.80):
	"""
	It calculates the Value at Risk (VaR) of some time series. It represents 
	the maximum loss with the given confidence level.
	
	Parameters
	----------
	returns : pandas.DataFrame
		Returns of each time serie. It could be daily, weekly, monthly, ...
		
	confidence_level : int
		Confidence level. 5% by default.
			
	Returns
	-------
	var : pandas.Series
		Value at Risk for each time series.
	
	"""
	
	# Calculating VaR
	returns = np.asarray(returns)
	return np.round(np.quantile(returns, confidence_level, axis=0, interpolation='higher'), 2)

In [ ]:
def expected_shortfall(returns, confidence_level=.80):
	"""
	It calculates the Expected Shortfall (ES) of some time series. It represents 
	the average loss according to the Value at Risk.
	
	Parameters
	----------
	returns : pandas.DataFrame
		Returns of each time serie. It could be daily, weekly, monthly, ...
		
	confidence_level : int
		Confidence level. 5% by default.
			
	Returns
	-------
	es : pandas.Series
		Expected Shortfall for each time series.
	
	"""
	
	# Calculating VaR
	returns = np.asarray(returns)
	var = value_at_risk(returns, confidence_level)
	
	# ES is the average of the worst losses (under var)
	return np.round(returns[np.greater(returns, var)].mean(), 2)  # adaptation to worst-case values (average of highest values)
	#return np.round(returns[np.less(returns, var)].mean(), 2)

In [ ]:
def var_80(a):
    return value_at_risk(a, confidence_level=.80)

In [ ]:
def var_90(a):
    return value_at_risk(a, confidence_level=.90)

In [ ]:
def var_95(a):
    return value_at_risk(a, confidence_level=.95)

In [ ]:
def var_99(a):
    return value_at_risk(a, confidence_level=.99)

In [ ]:
def cvar_80(a):
    return expected_shortfall(a, confidence_level=.80)

In [ ]:
def cvar_90(a):
    return expected_shortfall(a, confidence_level=.90)

In [ ]:
def cvar_95(a):
    return expected_shortfall(a, confidence_level=.95)

In [ ]:
def cvar_99(a):
    return expected_shortfall(a, confidence_level=.99)

### Sanity check: find duplicated rows

In [ ]:
df[df.duplicated(['InstanceName', 'Model', 'Strategy', 'Reoptimize', 'ModelPolicy', 'ForecastType', 'ModelName', 'RTCS_Policy', 'Gamma', 'GammaPerc', 'ScenarioId', 't', 'd'], keep=False)]

In [ ]:
df.drop_duplicates(subset=['InstanceName', 'Model', 'Strategy', 'Reoptimize', 'ModelPolicy', 'ForecastType', 'ModelName', 'RTCS_Policy', 'Gamma', 'GammaPerc', 'ScenarioId', 't', 'd'], 
                   keep='last', inplace=True)

### 3.1. Merge dataframe with scenario dataframe

In [ ]:
# Get the season start and end dates (northern hemisphere) for a specific year Y.
def get_season_dates(Y):
    return dict( {"winter" : [ (pd.Timestamp(year=Y, month=1, day=1, hour=0), pd.Timestamp(year=Y, month=3, day=20, hour=23, minute=59, second=59)), 
                              (pd.Timestamp(year=Y, month=12, day=21, hour=0), pd.Timestamp(year=Y, month=12, day=31, hour=23, minute=59, second=59))],
               "spring" : [(pd.Timestamp(year=Y, month=3, day=21, hour=0), pd.Timestamp(year=Y, month=6, day=20, hour=23, minute=59, second=59))],
               "summer" : [(pd.Timestamp(year=Y, month=6, day=21, hour=0), pd.Timestamp(year=Y, month=9, day=22, hour=23, minute=59, second=59))],
               "autumn" : [(pd.Timestamp(year=Y, month=9, day=23, hour=0), pd.Timestamp(year=Y, month=12, day=20, hour=23, minute=59, second=59))] })

In [ ]:
print("[Japan instances] Reading scenarios from dataframe files...")
instance_group = 'japan-10'
scenario_folder = os.path.join(instances_folder, "japan_microgrid", instance_group, "scenarios")
csv_filelist = []
for filename in glob.glob(os.path.join(scenario_folder, "*.csv.gz")):
    inputfile = os.path.join(scenario_folder, filename)
    csv_filelist.append((filename, inputfile))
# end
# Read all files and append to a single dataframe df
df_list = []
for filename, inputfile in csv_filelist:
    print('Reading file ', filename)
    df_s = pd.read_csv(filename)
    df_list.append(df_s)
# end
df_scenarios = pd.concat(df_list)
display(df_scenarios.info())
# convert timestamp column to datetime
df_scenarios['timestamp'] = pd.to_datetime(df_scenarios['timestamp'])  # DateTime.(df[!, :timestamp], dateformat"yyyy-mm-dd HH:MM:SS")
first_date = df_scenarios['timestamp'].min()
last_date = df_scenarios['timestamp'].max()
print(first_date, last_date)

In [ ]:
# Obtain instance season by extracting last part of the instance_name
df_dict_scenarios = dict()
for season in ['winter', 'spring', 'autumn', 'summer']:
    print("[read_scenario_dataframes] Filtering scenario dataframe with season criteria: season == ", season)
    df_list_seasonal = []
    first_year = df_scenarios['timestamp'].min().year
    last_year = df_scenarios['timestamp'].max().year
    for year in range(first_year, last_year+1):
        season_date_dict = get_season_dates(year)
        date_list = season_date_dict[season]
        for start_end_date_tuple in date_list:
            start_date = start_end_date_tuple[0]
            end_date = start_end_date_tuple[1]
            print("Season {}: [{} - {}]".format(season, start_date, end_date))
            df_season = df_scenarios[((df_scenarios['timestamp'] >= start_date) & (df_scenarios['timestamp'] <= end_date))]
            df_list_seasonal.append(df_season)
        # end
    # end
    df_dict_scenarios[season] = pd.concat(df_list_seasonal)
    df_dict_scenarios[season]['day'] = df_dict_scenarios[season]['timestamp'].dt.day
    unique_day_list = df_dict_scenarios[season]['timestamp'].dt.normalize().unique()
    print(len(unique_day_list), 'days in total')
    scenario_count = 0
    for unique_day in unique_day_list:
        scenario_count += 1
        df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.normalize() == unique_day), 'ScenarioId']  = scenario_count
    df_dict_scenarios[season]['t'] = df_dict_scenarios[season]['timestamp'].dt.hour + 1
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 0), 'd'] = 1
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 10), 'd'] = 2
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 20), 'd'] = 3
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 30), 'd'] = 4
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 40), 'd'] = 5
    df_dict_scenarios[season].loc[(df_dict_scenarios[season]['timestamp'].dt.minute == 50), 'd'] = 6
    df_dict_scenarios[season]['ScenarioId'] = df_dict_scenarios[season]['ScenarioId'].astype(int)
    df_dict_scenarios[season]['t'] = df_dict_scenarios[season]['t'].astype(int)
    df_dict_scenarios[season]['d'] = df_dict_scenarios[season]['d'].astype(int)
# end
display(df_dict_scenarios['winter'].head(10))
#display(df_dict_scenarios['winter'].info())
# df_dict_scenarios[season]['PV_Production_Wh']

### 3.2. Calculate Out Of Contract (OC) Cost

In [ ]:
def calculate_OC_cost(microgrid):
    if microgrid == 'utc':
        oc_cost = [0.0197817, 0.0197817, 0.0197817, 0.0197817, 0.0197817, 0.0197817, 0.025343, 0.040525, 0.0340525, 0.045779, 0.045779, 0.045779, 0.0385546, 0.0391196, 0.0390639, 0.0390639, 0.0390639, 0.0388545, 0.0420445, 0.0331963, 0.0268793, 0.037982, 0.0290351, 0.0227148]
    else:  # japan
        oc_cost = [0.01499000, 0.01499000, 0.01499000, 0.01499000, 0.01499000, 0.02998000, 0.02998000, 0.02998000, 0.06275000, 0.06275000, 0.02998000, 0.02998000, 0.02998000, 0.02998000, 0.02998000, 0.02998000, 0.02998000, 0.06275000, 0.06275000, 0.02998000, 0.02998000, 0.01499000, 0.01499000]
    d = {'t': [_ for _ in range(1, len(oc_cost)+1)], 'oc_unit_cost': oc_cost}
    df_oc = pd.DataFrame(data=d)
    return df_oc

In [ ]:
calculate_OC_cost('japan')

#### Calcular as estatisticas de state of charge das baterias, em cada instante de tempo t : Stored Avg

In [ ]:
df.describe()

In [ ]:
#from numba import jit
#@jit(nopython=True)
def calculate_each_soc(scenario_id, g_td, h_td, uInit, lost_coeff = 0.15):  # e.g. lost_coeff = 0.1 (10 %)
    soc = np.empty(h_td.shape[0])
    last_scenario_id = -1
    for i in range(0, h_td.shape[0]):
        if scenario_id[i] != last_scenario_id:
            soc[i] = uInit + (1 - lost_coeff) * (g_td[i]) - h_td[i]
        else:
            soc[i] = soc[i-1] + (1 - lost_coeff) * (g_td[i]) - h_td[i]
        last_scenario_id = scenario_id[i]
    return soc

In [ ]:
def calculate_battery_soc(df_b, uInit = 100000, uMax = 309700):
    df_b.sort_values(by=['ScenarioId', 't', 'd'], inplace=True)
    df_b['SOC'] = 100.0 * calculate_each_soc(*df_b[['ScenarioId', 'g_td', 'h_td']].values.T, uInit) / uMax  # df.loc[~((df['t'] == 1) & (df['d'] == 1))], df.loc[~((df['t'] == 1) & (df['d'] == 1)), ['SOC']].shift(-1))
    return df_b

#### Calcular o % de instantes de tempo t em que houve cobranca fora de contrato (OC)

In [ ]:
def calculate_penalty_freq(df_b):
    df_group = df_b.groupby(by=['ScenarioId', 't']).sum()
    total = len(df_group.index)
    df_occurence = df_group[df_group['e_td_x'] >= 0.1]
    num_occurence = len(df_occurence.index)
    return 100 * num_occurence / float(total)

#### Calcular o % de utilizacao da energia solar (vide instantes em que o gap > 0) : Renewables Util

In [ ]:
def calculate_renewables_util(df_b, season):
    df_plus_scenarios = df_b.merge(df_dict_scenarios[season], on=['ScenarioId', 't', 'd'])
    df_group = df_plus_scenarios.groupby(by=['ScenarioId', 't']).sum()
    df_group['min_gap_PV'] = df_group[['gap','PV_Production_Wh']].min(axis=1)
    df_group['Renewables Util'] = 100 * (df_group['gap'] <= 0).astype(int) + 100 * (df_group['gap'] > 0).astype(int) * (1.0 - (df_group['min_gap_PV'] / df_group['PV_Production_Wh']))
    return df_group

## 3.3. Simulation snapshot graphs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_oc_cost = calculate_OC_cost('japan')
# multiplicando o consumo pelos custos OC para cada periodo t, em separado
df_with_oc_cost = df.merge(df_oc_cost, on=['t'])

In [ ]:
df_with_oc_cost['oc_cost'] = df_with_oc_cost['oc_unit_cost'] * df_with_oc_cost['e_td_x']

In [ ]:
def plot_OC_for_scenario(instance_name, df_dict_scenarios, df, which_scenario, 
                         strategy, gamma_det, gamma_rob):
    df_ = df.copy()
    df_ = df_[(df_['InstanceName'] == instance_name)]
    df_ = df_[(df_['RTCS_Policy'] == strategy)]
    #df_ = df_[(df_['ScenarioId'] == which_scenario)]
    # Filter out scenarios where OC-cost == 0
    oc_cost_det = df_[(df_['Model'] == 'deterministic') & (df_['GammaPerc'] == gamma_det)]['e_td_x'].sum()
    if oc_cost_det <= 1.0:
        display(f'Skipping scenario {which_scenario}, since sum-OC-cost-det == 0.')
        return
    else:
        display(f'Plot of scenario {which_scenario}. sum-OC-cost-det = {oc_cost_det}')
    
    df_scenario_info = df_dict_scenarios[instance_name]
    df_scenario_info = df_scenario_info[(df_scenario_info['ScenarioId'] == which_scenario)]

    df_det = df_[(df_['Model'] == 'deterministic')]

    # merge the results df with the instance scenario info df
    df_all = df_scenario_info.merge(df_, how='inner', on=['ScenarioId', 't', 'd'])

    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(11.7, 8.27)
    df_scenario_info.plot(x="timestamp", y="PV_Production_Wh", ax=ax, legend=False, color="b")
    df_scenario_info.plot(x="timestamp", y="Building_Consumption_Wh", ax=ax, legend=False, color="g")

    ax2 = ax.twinx()

    df_det = df_all[(df_all['Model'] == 'deterministic')]
    #display(df_det['GammaPerc'].unique().tolist())
    df_det = df_det[(df_det['GammaPerc'] == gamma_det)].rename(columns={"e_td_x": "OC-det"})
    #display('Det')
    #display(df_det['OC-det'])
    df_det.plot(x="timestamp", y="OC-det", ax=ax2, legend=False, color="r")

    df_rob = df_all[(df_all['Model'] == 'robust-budget')]
    #display(df_rob['GammaPerc'].unique().tolist())
    df_rob = df_rob[(df_rob['GammaPerc'] == gamma_rob)].rename(columns={"e_td_x": "OC-rob"})
    #display('Rob')
    #display(df_rob.head())
    df_rob.plot(x="timestamp", y="OC-rob", ax=ax2, legend=False, color="y")

    ax.figure.legend()
    plt.show()

In [ ]:
#instance_name, gamma_det, gamma_rob = 'summer', 50, 40
#instance_name, gamma_det, gamma_rob = 'autumn', 50, 60

strategy = 'cheapest+LA+ReOpt'
df_temp = df_with_oc_cost[(df_with_oc_cost['InstanceName'] == instance_name)]
df_temp = df_temp[(df_temp['RTCS_Policy'] == strategy)]

In [ ]:
def plot_higher_det_cost_for_scenario(df_oc_cost, df_dict_scenarios, df, which_scenario, gamma_det, gamma_rob,
                                     start_time = None, end_time = None, cum_sum = False, plot_oc = True):
    df_ = df.copy()
    
    df_ = df_[(df_['ScenarioId'] == which_scenario)]
    # Filter out scenarios where OC-cost == 0
    df_oc_cost_det = df_[(df_['Model'] == 'deterministic') & (df_['GammaPerc'] == gamma_det)]
    max_oc_cost_det = df_oc_cost_det['oc_cost'].max()
    sum_oc_cost_det = df_oc_cost_det['oc_cost'].sum()
    if sum_oc_cost_det <= 1.0:
        display(f'Skipping scenario {which_scenario}, since sum-OC-cost-det == 0.')
        return
    else:
        display(f'Plot of scenario {which_scenario}. sum-OC-cost-det = {sum_oc_cost_det} and max_oc_cost_det = {max_oc_cost_det}.')
        
    # Filter out scenarios, selecting those where det cost went higher than rob-cost at least once
    cost_det = df_[(df_['Model'] == 'deterministic') & (df_['GammaPerc'] == gamma_det)]
    cost_rob = df_[(df_['Model'] == 'robust-budget') & (df_['GammaPerc'] == gamma_rob)]
    cost_all = cost_det.merge(cost_rob, how='inner', on=['ScenarioId', 't', 'd'])
    cost_all['det_higher'] = (cost_all['cost_x'] > cost_all['cost_y']).astype(int)
    sum_high_cost_det = cost_all['det_higher'].sum()
    if sum_high_cost_det <= 1.0:
        display(f'Skipping scenario {which_scenario}, since sum_high_cost_det == 0.')
        return
    else:
        display(f'Plot of scenario {which_scenario}. sum_high_cost_det = {sum_high_cost_det}')
    
    df_scenario_info = df_dict_scenarios[instance_name]
    df_scenario_info = df_scenario_info[(df_scenario_info['ScenarioId'] == which_scenario)]

    # merge the results df with the instance scenario info df
    df_all = df_scenario_info.merge(df_, how='inner', on=['ScenarioId', 't', 'd'])
    if start_time is not None:
        df_all.set_index(['timestamp'], inplace=True)
        df_all = df_all.between_time(start_time, end_time)
        df_all.reset_index(inplace=True)

    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(15.7, 8.27)
    #df_scenario_info.plot(x="timestamp", y="PV_Production_Wh", ax=ax, legend=False, color="b", marker='+')
    if cum_sum:
        df_scenario_info['Building_Consumption_Wh_cumsum'] = df_scenario_info['Building_Consumption_Wh'].cumsum()
        df_scenario_info.plot(x="timestamp", y="Building_Consumption_Wh_cumsum", ax=ax, legend=False, color="g", marker='^')
    else:
        df_scenario_info.plot(x="timestamp", y="Building_Consumption_Wh", ax=ax, legend=False, color="g", marker='^')

    ax2 = ax.twinx()

    df_det = df_all[(df_all['Model'] == 'deterministic')]
    #display(df_det['GammaPerc'].unique().tolist())
    df_det = df_det[(df_det['GammaPerc'] == gamma_det)].rename(columns={"cost": "cost-det", "oc_cost": "OC-det"})
    ###display('Det')
    ###display(df_det['OC-det'])

    df_rob = df_all[(df_all['Model'] == 'robust-budget')]
    #display(df_rob['GammaPerc'].unique().tolist())
    df_rob = df_rob[(df_rob['GammaPerc'] == gamma_rob)].rename(columns={"cost": "cost-rob", "oc_cost": "OC-rob"})
    ###display('Rob')
    ###display(df_rob.head())
    if cum_sum:
        label_det = "-det-cum"
        label_rob = "-rob-cum"
        df_det["cost-det-cum"] = df_det["cost-det"].cumsum()
        df_rob["cost-rob-cum"] = df_rob["cost-rob"].cumsum()
        df_det["OC-det-cum"] = df_det["OC-det"].cumsum()
        df_rob["OC-rob-cum"] = df_rob["OC-rob"].cumsum()
    else:
        label_det = "-det"
        label_rob = "-rob"
    # end if
    df_det.plot(x="timestamp", y='cost'+label_det, ax=ax2, legend=False, color="r", marker='>')
    df_rob.plot(x="timestamp", y='cost'+label_rob, ax=ax2, legend=False, color="y", marker='x')
    if plot_oc:
        df_det.plot(x="timestamp", y="OC"+label_det, ax=ax2, legend=False, color="purple", marker='o')
        df_rob.plot(x="timestamp", y="OC"+label_rob, ax=ax2, legend=False, color="grey", marker='o')
    # end if
    
    ax.grid('on', which='minor', axis='x' )
    ax.grid('on', which='major', axis='x' )
    ax.grid('on', which='minor', axis='y' )
    ax.grid('on', which='major', axis='y' )
    

    ax.figure.legend()
    plt.show()
    del df_
    del cost_det
    del cost_rob
    del cost_all
    del df_scenario_info
    del df_det
    del df_rob
    

In [ ]:
# scenario 170 - 9, 9
# scenario 181 - 6, 12 - 'autumn', 50, 40
# sc 197 - 07:00 - 09:00
plot_higher_det_cost_for_scenario(df_oc_cost, df_dict_scenarios, df_temp, 181, gamma_det, gamma_rob, '07:00', '10:30',
                                 cum_sum = False)

In [ ]:
# scenario 170 - 9, 9
# scenario 181 - 6, 12 - 'autumn', 50, 40
# sc 197 - 07:00 - 09:00
plot_higher_det_cost_for_scenario(df_oc_cost, df_dict_scenarios, df_temp, 181, gamma_det, gamma_rob, 
                                 cum_sum = True)

## 3.4. Accumulated costs after all scenario runs

In [ ]:
def plot_costs_all_scenarios(df_all, cum_sum = False, plot_oc = True):
    
    
    #df_all = df_all[(df_all['timestamp'].dt.year == year)]
    df_all = df_all.sort_values(by=['timestamp'])
    df_all['Building_Consumption (Wh)'] = df_all['Building_Consumption_Wh'].cumsum()
    
    fig, ax = plt.subplots()
    ax2 = ax.twinx()
    # the size of A4 paper
    fig.set_size_inches(15.7, 8.27)
    plot = True
    if plot:
        df_all.plot(x="timestamp", y="Building_Consumption (Wh)", ax=ax2, legend=False, color="g") #, marker='^')
        #df_all.plot(x="timestamp", y="PV_Production_Wh", ax=ax, legend=False, color="b")  #, marker='+')

    #df_all = df_all.sort_values(by=['timestamp'])
    df_det = df_all[df_all['Model'] == 'deterministic'].sort_values(by=['timestamp'])
    df_det['OC-Cost-Det (Euros)'] = df_det['OC-det'].cumsum()
    df_det['Total-Cost-Det (Euros)'] = df_det['cost-det'].cumsum()
    df_rob = df_all[df_all['Model'] == 'robust-budget'].sort_values(by=['timestamp'])
    df_rob['OC-Cost-Rob (Euros)'] = df_rob['OC-rob'].cumsum()
    df_rob['Total-Cost-Rob (Euros)'] = df_rob['cost-rob'].cumsum()
    
    label_det = "-Det_Euros"
    label_rob = "-Rob_Euros"
    df_det.plot(x="timestamp", y='Total-Cost-Det (Euros)', ax=ax, legend=False, color="r") #, marker='>')
    df_rob.plot(x="timestamp", y='Total-Cost-Rob (Euros)', ax=ax, legend=False, color="y") #, marker='x')
    if plot_oc:
        df_det.plot(x="timestamp", y='OC-Cost-Det (Euros)', ax=ax, legend=False, color="purple") #, marker='o')
        df_rob.plot(x="timestamp", y='OC-Cost-Rob (Euros)', ax=ax, legend=False, color="grey") #, marker='o')
    # end if
    
    ax.grid('on', which='minor', axis='x' )
    ax.grid('on', which='major', axis='x' )
    ax.grid('on', which='minor', axis='y' )
    ax.grid('on', which='major', axis='y' )
    
    ax.set_xlabel("Scenario (Date)")
    ax2.set_ylabel('Energy (Wh)')
    ax.set_ylabel('Cost (Euros)')

    #ax.figure.legend()
    ax.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.show()
    fig.savefig('cumulative_costs_time_horizon.pdf')
    print('Final costs:')
    print('Det:\n', df_det.iloc[-1])
    print('Rob:\n', df_rob.iloc[-1])
    del df_det
    del df_rob
    

In [ ]:
instance_gamma_dict = {'summer' : (50, 40), 'winter' : (100, 100), 'autumn' : (50, 40), 'spring' : (50, 40)}
df_list = []
strategy = 'cheapest+LA+ReOpt'
df_all = df_with_oc_cost[(df_with_oc_cost['RTCS_Policy'] == strategy)]
for instance_name in ['summer', 'winter', 'autumn', 'spring']:
    df_scenario_info = df_dict_scenarios[instance_name]
    
    df_det = df_all[(df_all['Model'] == 'deterministic') & (df_all['InstanceName'] == instance) & (df_all['GammaPerc'] == instance_gamma_dict[instance][0])]
    df_det = df_det.rename(columns={"cost": "cost-det", "oc_cost": "OC-det"})
    df_det = df_scenario_info.merge(df_det, how='inner', on=['ScenarioId', 't', 'd'])
    df_list.append(df_det)
    
    df_rob = df_all[(df_all['Model'] == 'robust-budget') & (df_all['InstanceName'] == instance) & (df_all['GammaPerc'] == instance_gamma_dict[instance][1])]
    df_rob = df_rob.rename(columns={"cost": "cost-rob", "oc_cost": "OC-rob"})
    df_rob = df_scenario_info.merge(df_rob, how='inner', on=['ScenarioId', 't', 'd'])
    df_list.append(df_rob)
# end for
df_temp = pd.concat(df_list)


In [ ]:
plot_costs_all_scenarios(df_temp, cum_sum = True, plot_oc = True)

## Table 0. Number of scenarios per instance

In [ ]:
df_num_scenarios_per_instance = df[((df['t'] == 1) & (df['d'] == 1))].groupby(by=['InstanceName', 'Model', 'ModelName', 'RTCS_Policy', 'GammaPerc']).count().reset_index()
df_num_scenarios_per_instance = df_num_scenarios_per_instance[['InstanceName', 'Model', 'ModelName', 'RTCS_Policy', 'Gamma', 'GammaPerc', 'ScenarioId']]
df_num_scenarios_per_instance.rename(columns={'ScenarioId' : 'ScenarioCount'}, inplace=True)
df_num_scenarios_per_instance

## Table 1. Simulation performance given all instances 

Model-wise RTCS simulation performance comparison, given all instances.

* Median, Mean, Std. dev and sum of each measure (cost, e_td, gap, time). 

In [ ]:
# https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
import numpy as np, scipy.stats as st

def compute_ci(a, conf=0.95):
    return [np.round(_, 2) for _ in st.t.interval(conf, len(a)-1, loc=np.mean(a), scale=st.sem(a))]

def compute_iqr(a):
    return st.iqr(a)

In [ ]:
per_instance_stats = dict()
instances_to_process = df['InstanceName'].unique().tolist()
for instance_name in instances_to_process:  # group by instance
    df_itype = df[(df['InstanceName'] == instance_name)]
    _model_list = df_itype['Model'].unique().tolist()
    for model in _model_list:
        df_model = df_itype[df_itype['Model'] == model]
        _gamma_perc_list = df_model['GammaPerc'].unique().tolist()
        for gamma_perc in _gamma_perc_list:
            df_gamma = df_model[df_model['GammaPerc'] == gamma_perc]
            policy_list = df_gamma['RTCS_Policy'].unique().tolist()
            for policy in policy_list:
                  df_ = df_gamma[df_gamma['RTCS_Policy'] == policy]
                  reopt = df_['Reoptimize'].iloc[0]
                  strategy = df_['Strategy'].iloc[0]
                  df_oc_cost = calculate_OC_cost('japan')
                  # multiplicando o consumo pelos custos OC para cada periodo t, em separado
                  df_ = df_.merge(df_oc_cost, on=['t'])
                  df_['oc_cost'] = df_['oc_unit_cost'] * df_['e_td_x']
                  df_ungrouped = df_.copy()
                  df_ = df_.groupby(by=['ScenarioId']).sum()
                  key = (instance_name, model, gamma_perc, policy, strategy, reopt)
                  per_instance_stats[key] = dict()
                  #per_instance_stats[key]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
                  #per_instance_stats[key]['% Solved'] = calculate_perc_solved(df_rpfs, model, instance_type, instance_size)
                  #per_instance_stats[key]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
                  per_instance_stats[key]['Median time (s)'] = np.round(df_['RealProcTime'].median(), 2)
                  per_instance_stats[key]['Avg. time (s)'] = np.round(df_['RealProcTime'].mean(), 2)
                  per_instance_stats[key]['Std. dev. of time (s)'] = np.round(df_['RealProcTime'].std(), 2)
                  per_instance_stats[key]['Total time (s)'] = np.round(df_['RealProcTime'].sum(), 2)
                  
                  per_instance_stats[key]['Median cost ($)'] = np.round(df_['cost'].median(), 2)
                  per_instance_stats[key]['Cost Avg ($)'] = np.round(df_['cost'].mean(), 2)
                  per_instance_stats[key]['Cost Std ($)'] = np.round(df_['cost'].std(), 2)
                  per_instance_stats[key]['Cost Total (M$)'] = np.round(df_['cost'].sum() / 1000.0, 2)
                  per_instance_stats[key]['Cost Max ($)'] = np.round(df_['cost'].max(), 2)
                  per_instance_stats[key]['Cost VaR 80% ($)'] = np.round(var_80(df_['cost']), 2)
                  per_instance_stats[key]['Cost VaR 90% ($)'] = np.round(var_90(df_['cost']), 2)
                  per_instance_stats[key]['Cost VaR 95% ($)'] = np.round(var_95(df_['cost']), 2)
                  per_instance_stats[key]['Cost VaR 99% ($)'] = np.round(var_99(df_['cost']), 2)
                  per_instance_stats[key]['Cost CVaR 80% ($)'] = np.round(cvar_80(df_['cost']), 2)
                  per_instance_stats[key]['Cost CVaR 90% ($)'] = np.round(cvar_90(df_['cost']), 2)
                  per_instance_stats[key]['Cost CVaR 95% ($)'] = np.round(cvar_95(df_['cost']), 2)
                  per_instance_stats[key]['Cost CVaR 99% ($)'] = np.round(cvar_99(df_['cost']), 2)
                    
                  per_instance_stats[key]['Cost IQR ($)'] = compute_iqr(df_['cost'])
                  
                  per_instance_stats[key]['Median gap (kWh)'] = np.round(df_['gap'].median(), 2)
                  per_instance_stats[key]['Avg. gap (kWh)'] = np.round(df_['gap'].mean(), 2)
                  per_instance_stats[key]['Std. dev. of gap (kWh)'] = np.round(df_['gap'].std(), 2)
                  per_instance_stats[key]['Total gap (kWh)'] = np.round(df_['gap'].sum(), 2)
                  
                  per_instance_stats[key]['OC Cost Median ($)'] = np.round(df_['oc_cost'].median(), 2)
                  per_instance_stats[key]['OC Cost Avg ($)'] = np.round(df_['oc_cost'].mean(), 2)
                  per_instance_stats[key]['OC Cost Std ($)'] = np.round(df_['oc_cost'].std(), 2)
                  per_instance_stats[key]['OC Cost Total ($)'] = np.round(df_['oc_cost'].sum(), 2)
                
                  # Calcular o % de instantes de tempo t em que houve cobranca fora de contrato (OC)
                  per_instance_stats[key]['Penalty Freq (%)'] = calculate_penalty_freq(df_ungrouped)
                
                  # Calcular as estatisticas de state of charge das baterias, em cada instante de tempo t : Stored Avg
                  df_soc = calculate_battery_soc(df_ungrouped)
                  #df_soc = df_ungrouped.copy()
                  #df_soc['SOC'] = 0
                  per_instance_stats[key]['SOC Median (%)'] = np.round(df_soc['SOC'].median(), 2)
                  per_instance_stats[key]['SOC Avg (%)'] = np.round(df_soc['SOC'].mean(), 2)
                  per_instance_stats[key]['SOC Std (%)'] = np.round(df_soc['SOC'].std(), 2)
                  per_instance_stats[key]['SOC CI (%)'] = compute_ci(df_soc['SOC'])
                  per_instance_stats[key]['SOC IQR (%)'] = compute_iqr(df_soc['SOC'])
                  #per_instance_stats[key]['Stored Total'] = np.round(df_soc['SOC'].sum(), 2)
                    
                  # Calcular o % de utilizacao da energia solar (vide instantes em que o gap > 0) : Renewables Util
                  df_ren = calculate_renewables_util(df_ungrouped, instance_name)
                  per_instance_stats[key]['Renewables Util Median (%)'] = np.round(df_ren['Renewables Util'].median(), 2)
                  per_instance_stats[key]['Renewables Util Avg (%)'] = np.round(df_ren['Renewables Util'].mean(), 2)
                  per_instance_stats[key]['Renewables Util Std (%)'] = np.round(df_ren['Renewables Util'].std(), 2)
                  per_instance_stats[key]['Renewables Util IQR (%)'] = compute_iqr(df_ren['Renewables Util'])
                  #per_instance_stats[key]['Renewables Util Total'] = np.round(df_ren['Renewables Util'].sum(), 2)
                

In [ ]:
df_table1 = pd.DataFrame.from_dict(per_instance_stats)
df_table1b = df_table1.T.reset_index()
df_table1b.rename(columns={'level_0': 'Instance', 'level_1' : 'Model', 'level_2' : 'Gamma', 'level_3' : 'RTCS Policy',
                          'level_4' : 'Strategy', 'level_5' : 'Reoptimize'}, inplace=True)
df_table1b.head(12)
df_table1b.to_pickle(os.path.join(reportfolder, 'df_japan_table1.pkl.gz'))

In [ ]:
df_table1b.info()

In [ ]:
id_vars = ['Instance', 'Model', 'Gamma', 'RTCS Policy', 'Strategy', 'Reoptimize']
value_vars = [_ for _ in df_table1b.columns if _ not in id_vars]
df_stats = pd.melt(df_table1b, id_vars=id_vars, value_vars=value_vars, value_name='value')
df_stats.to_csv(os.path.join(reportfolder_table, 'japan-stats.csv'))
#df_stats.to_excel(os.path.join(reportfolder_table, 'japan-stats.xlsx'))
print('Saved to folder', reportfolder)
df_stats.tail(10)

## Plotting Pareto front: cost and std average obtained by different models (several Gamma values)

In [ ]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [ ]:
for season in ['winter', 'spring', 'autumn', 'summer']:
    df_pareto = df_table1b[(df_table1b['Instance'] == season)] # & (df_table1b['RTCS Policy'] == 'cheapest+LA+ReOpt')]
    df_pareto = df_pareto[(df_pareto['Model'] != 'robust-box')]
    df_pareto = df_pareto[(df_pareto['Strategy'] != 'audacious')]
    policy_list = ['conservative+LA+ReOpt', 'conservative+LA', 'cheapest+LA+ReOpt', 'cheapest+LA', 'naive']
    df_pareto = df_pareto[(df_pareto['RTCS Policy'].isin(policy_list))]
    df_pareto['RTCS Policy'] = df_pareto['RTCS Policy'].replace({'conservative+LA+ReOpt': 'conservative+ReOpt', 
                                          'conservative+LA': 'conservative', 
                                          'cheapest+LA+ReOpt': 'cheapest+ReOpt',
                                          'cheapest+LA': 'cheapest'
                                         })
    df_pareto.rename(columns={'Cost Std ($)' : 'Cost std over all simulated scenarios (Euros)',
                              'Cost Avg ($)' : 'Cost average over all simulated scenarios (Euros)'
                             }, inplace=True)

    df_pareto['Model / RTCS Policy'] = df_pareto['Model'] + ' / ' + df_pareto['RTCS Policy'].astype(str)
    #display(df_pareto[['Instance', 'Model', 'RTCS Policy', 'Gamma', 'Cost Std ($)', 'Cost Avg ($)']])
    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(10.7, 8.27)
    fig.suptitle(f'Pareto front - {season}', fontsize=20)
    #linestyle = [linestyle_tuple[i][1] for i in range(len(linestyle_tuple))]
    linestyle = ['--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', '-.',    '--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', '-.']
    markers = ['d', '>', '<', 'x', '^', 'v', 's', 'o', 'D',  '*', '+', 'o', 'x', '^', '8', 's', 'p', 'D']
    #marker = marker[::-1]
    linewidth = 1.6
    with sns.axes_style("whitegrid"):
        sns.lineplot(ax=ax, data=df_pareto, x='Cost std over all simulated scenarios (Euros)', 
                     y='Cost average over all simulated scenarios (Euros)', hue='Model / RTCS Policy', 
                     style='Model / RTCS Policy',
                     #linestyle=linestyle, 
                     markers=True)
        sns.set_context("paper", font_scale=1.2) #, rc={"grid.linewidth": 1, "lines.linewidth": linewidth})
        fig.savefig(f'pareto_{season}.pdf')
# end for

#g = sns.FacetGrid(df_pareto, hue="Model", size=8)
#g.map(plt.scatter, 'Cost Std ($)', 'Cost Avg ($)')
#g.map(plt.plot, 'Cost Std ($)', 'Cost Avg ($)')
#plot_pareto_front_all_policies(df_pareto)

### Table 2. Total cost considering all simulations for a specific CCP model and RTCS policy

In [ ]:
df_totals = df.drop(columns=['t', 'd', 'OptTimeSpent']).groupby(by=['InstanceName', 'Model', 'GammaPerc', 'Gamma', 'RTCS_Policy']).sum()
df_total_proc_time = df_totals.drop(columns=['ScenarioId', 'e_td_x', 'gap', 'ObjValue', 'cost'])
df_total_cost = df_totals.drop(columns=['ScenarioId', 'e_td_x', 'gap', 'ObjValue', 'RealProcTime']).reset_index()
# total simulation cost of the deterministic model
df_total_cost_det = df_total_cost[(df_total_cost['Model'] == 'deterministic')].drop(columns=['Model', 'GammaPerc', 'Gamma']).rename(columns={"cost": "cost(det)"})
# total simulation cost of the box model
df_total_cost_box = df_total_cost[(df_total_cost['Model'] == 'robust-box')].drop(columns=['Model', 'GammaPerc', 'Gamma']).rename(columns={"cost": "cost(box)"})
# total simulation cost of the budget model
df_total_cost_bud = df_total_cost[(df_total_cost['Model'] == 'robust-budget')].drop(columns=['Model']).rename(columns={"cost": "cost(bud)"})
df_total_cost_bud_pivot = pd.pivot_table(df_total_cost_bud, values='cost(bud)', index=['InstanceName', 'RTCS_Policy'], \
                                         columns=['GammaPerc'], aggfunc=np.sum)
df_total_cost_bud_pivot.columns = [('Cost(bud_' + str(_) + ')') for _ in df_total_cost_bud_pivot.columns]
df_total_cost_bud_pivot = df_total_cost_bud_pivot.reset_index()


In [ ]:
df_total_cost_bud_pivot

#### Join the det, box and bud costs in the same dataframe for comparison

In [ ]:
join_columns_total_cost = ['InstanceName', 'RTCS_Policy']
df_total_cost_join = df_total_cost_det.merge(df_total_cost_bud_pivot, on=join_columns_total_cost, suffixes=('', '_bud'))
#.merge(df_total_cost_box, on=join_columns_total_cost, suffixes=('_det', '_box'))\
#df_total_cost_join.loc[(), 'Gamma'] = np.nan
#df_total_cost_join.loc[(), 'GammaPerc'] = np.nan
df_total_cost_join

### Table 3. Cost of the most expensive scenario (worst simulation cost), grouped by CCP model and simulation parameters

In [ ]:
df_t2 = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue'])
df_t2 = df_t2.groupby(by=['InstanceName', 'Model', 'ModelName', 'RTCS_Policy', 'ScenarioId']).sum().\
    drop(columns=['e_td_x', 'gap', 'RealProcTime', 'GammaPerc', 'Gamma']).\
    groupby(by=['InstanceName', 'Model', 'ModelName', 'RTCS_Policy']).\
    max()

df_rob = df_t2.reset_index()
df_rob = df_rob[(df_rob['Model'] == 'robust-budget') | (df_rob['Model'] == 'robust-box')]
df_det = df_t2.reset_index().drop(columns=['ModelName'])
df_det = df_det[df_det['Model'] == 'deterministic']
df_wins_t2 = df_rob.merge(df_det, on=['InstanceName', 'RTCS_Policy'], suffixes=('_rob', '_det'))\
    .drop(columns=['Model_det'])
df_wins_t2['MaxRobCost_Smaller'] = (df_wins_t2['cost_rob'] < df_wins_t2['cost_det']).astype(int)

In [ ]:
#p = sns.countplot(data=df_wins,
#                  y = 'InstanceName',
#                  hue = 'Model_rob')
# grouped barplot
# g = sns.barplot(x="ModelName", y="rob_wins", hue="InstanceName", data=df_wins_t2, ci=None)
g = sns.catplot(y="ModelName", x="MaxRobCost_Smaller",
                 col="InstanceName", hue="RTCS_Policy", 
                 palette="pastel", edgecolor=".6", # orient="h", height=1.5, aspect=4, 
                 data=df_wins_t2, kind="bar", ci=None)
g.set_xticklabels(rotation=90)

In [ ]:
df_wins_t2.set_index(['InstanceName', 'ModelName', 'RTCS_Policy']).head()

### Table 4. RTCS performance map (robust wins)

Number of scenarios where Robust RTCS obtained smaller cost, when compared to the Deterministic RTCS, when simulating the same scenario.

In [ ]:
df_scenario = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue'])
df_scenario['ModelName'] = df_scenario['Model']
df_scenario.loc[(df_scenario['Model'] == 'robust-budget'), 'ModelName'] = df_scenario.loc[(df_scenario['Model'] == 'robust-budget'), 'Model'] + '-'\
    + df_scenario.loc[(df_scenario['Model'] == 'robust-budget'), 'GammaPerc'].astype(str)
df_scenario = df_scenario.groupby(by=['InstanceName', 'Model', 'ModelName', 'RTCS_Policy', 'ScenarioId']).sum()\
    .drop(columns=['gap', 'RealProcTime', 'GammaPerc', 'Gamma']).reset_index()

# simulation cost of the deterministic model, per scenario
df_cost_det = df_scenario[(df_scenario['Model'] == 'deterministic')]
# simulation cost of the box model, per scenario
df_cost_box = df_scenario[(df_scenario['Model'] == 'robust-box')]
# simulation cost of the budget model, per scenario
df_cost_bud = df_scenario[(df_scenario['Model'] == 'robust-budget')]

df_t3 = pd.concat([df_cost_det, df_cost_box, df_cost_bud])
df_cheapest_policy_per_scenario = df_t3.drop(columns=['e_td_x', 'e_td_y', 'Reoptimize']).groupby(by=['InstanceName', 'ScenarioId']).min()
df_cheapest_policy_per_scenario.head()

In [ ]:
g = sns.catplot(x="ModelName", 
                 col="InstanceName", hue="RTCS_Policy",
                 data=df_cheapest_policy_per_scenario.reset_index(), kind="count", ci=None)
g.set_xticklabels(rotation=90)

In [ ]:
df_target = df_scenario.reset_index().drop(columns=['e_td_x', 'e_td_y'])
df_target = df_target[(df_target['Model'] == 'robust-budget') | (df_target['Model'] == 'robust-box')]
df_det = df_scenario.reset_index().drop(columns=['e_td_x', 'e_td_y', 'ModelName'])
df_det = df_det[df_det['Model'] == 'deterministic']
df_wins_t3 = df_target.merge(df_det, on=['InstanceName', 'RTCS_Policy', 'ScenarioId'], suffixes=('_target', '_det'))\
    .drop(columns=['Model_det', 'Model_target'])
df_wins_t3['Cost_Smaller'] = (df_wins_t3['cost_target'] <= df_wins_t3['cost_det']).astype(int)
df_wins_t3['Perc_Cost_Diff'] = np.round(100 * (df_wins_t3['cost_target'] - df_wins_t3['cost_det']) / df_wins_t3['cost_det'], 2)
df_wins_t3.head()

In [ ]:
df_wins_t3_grouped = df_wins_t3.groupby(by=['InstanceName', 'ModelName', 'RTCS_Policy']).sum()  # 'ForecastType'
df_wins_t3_grouped_perc = df_wins_t3_grouped.reset_index().merge(df_num_scenarios_per_instance, on=['InstanceName', 'ModelName', 'RTCS_Policy'])
df_wins_t3_grouped_perc['Cost_Smaller_Perc'] = np.round((100 * df_wins_t3_grouped_perc['Cost_Smaller']) / df_wins_t3_grouped_perc['ScenarioCount'], 0).astype(int)
df_wins_t3_grouped_perc.head()

In [ ]:
g = sns.catplot(x="ModelName", y="Cost_Smaller_Perc",
                 col="InstanceName", hue="RTCS_Policy",
                 data=df_wins_t3_grouped_perc.reset_index(), kind="bar", ci=None, orient='v')
g.set_xticklabels(rotation=90)

### Table 5. Average % difference in solution cost (det vs. others)

In [ ]:
df_wins_t4_grouped = df_wins_t3.groupby(by=['InstanceName', 'ModelName', 'RTCS_Policy']).agg({'Perc_Cost_Diff' : ['mean', 'std']})
df_wins_t4_grouped.columns = ['_'.join(t) for t in df_wins_t4_grouped.columns]
df_wins_t4_grouped.head()

In [ ]:
g = sns.catplot(x="ModelName", y="Perc_Cost_Diff_mean",
                 col="InstanceName", hue="RTCS_Policy",
                 data=df_wins_t4_grouped.reset_index(), kind="bar", ci=None, orient='v')
g.set_xticklabels(rotation=90)

### Table 6. Cheapest RTCS Strategy, per instance, model type and Gamma parameter

In [ ]:
group_key = ['InstanceName', 'Model', 'GammaPerc', 'RTCS_Policy', 'ScenarioId']
df_group = df.drop(columns=['t', 'd', 'OptTimeSpent', 'ObjValue']).groupby(by=group_key).sum()\
    .drop(columns=['gap', 'RealProcTime'])
display(df_group.head())
# Find the cheapest strategy for each model type
df_cheapest = df_group.groupby(by=['InstanceName', 'Model', 'GammaPerc', 'ScenarioId']).min().drop(columns=['e_td_x', 'Reoptimize'])
df_cheapest.head()

In [ ]:
# Get the number of scenarios per test
df_num_scenarios = df_cheapest.reset_index().groupby(by=['InstanceName', 'Model', 'GammaPerc']).count()[['ScenarioId']]
df_num_scenarios = df_num_scenarios.reset_index().rename(columns={"ScenarioId": "num_scenarios"})
#df_num_scenarios.to_excel(os.path.join(reportfolder_table, 'japan-num_scenarios-per-instance.xlsx'))
df_num_scenarios.to_csv(os.path.join(reportfolder_table, 'japan-num_scenarios-per-instance.csv'))
df_num_scenarios.head(15)

#### Scenario-wise cheapest strategy comparison

In [ ]:
df_target = df_cheapest.reset_index()
# Get the information about the cheapest strategies, per instance, model, gamma and scenario
df_wins_cheapest = df_target.merge(df_group.reset_index(), on=['InstanceName', 'Model', 'GammaPerc', 'ScenarioId'], suffixes=('_cheapest', ''))
df_wins_cheapest['wins'] = (np.abs((df_wins_cheapest['cost'] - df_wins_cheapest['cost_cheapest'])/df_wins_cheapest['cost_cheapest']) < 0.001).astype(int)
df_wins_per_instance_model_gamma_policy = df_wins_cheapest.groupby(by=['InstanceName', 'Model', 'GammaPerc', 'RTCS_Policy']).sum()[['wins']]
df_wins_per_instance_model_gamma_policy.rename(columns={"wins": "scenario_wins"}, inplace=True)
#df_wins_per_instance_model_gamma_policy.reset_index().to_excel(os.path.join(reportfolder_table, 'japan-df_wins_per_instance_model_gamma_policy.xlsx'))
df_wins_per_instance_model_gamma_policy.reset_index().to_csv(os.path.join(reportfolder_table, 'japan-df_wins_per_instance_model_gamma_policy.csv'))
display(df_wins_per_instance_model_gamma_policy)
df_perc_wins_per_instance_model_gamma_policy = df_wins_per_instance_model_gamma_policy.reset_index().merge(df_num_scenarios, on=['InstanceName', 'Model', 'GammaPerc'])
df_perc_wins_per_instance_model_gamma_policy['perc_scenario_wins'] = 100.0 * df_perc_wins_per_instance_model_gamma_policy['scenario_wins'] / df_perc_wins_per_instance_model_gamma_policy['num_scenarios']
#df_perc_wins_per_instance_model_gamma_policy.to_excel(os.path.join(reportfolder_table, 'japan-df_perc_wins_per_instance_model_gamma_policy.xlsx'))
df_perc_wins_per_instance_model_gamma_policy.to_csv(os.path.join(reportfolder_table, 'japan-df_perc_wins_per_instance_model_gamma_policy.csv'))
df_perc_wins_per_instance_model_gamma_policy

In [ ]:
df_wins_grouped = df_wins_cheapest.groupby(by=['InstanceName', 'Model_target', 'GammaPerc_target']).sum().drop(columns=['ScenarioId'])
df_wins_grouped['rob_wins_%'] = np.round(100 * df_wins_grouped['rob_wins'] / df_wins_grouped['#scenarios'], 2)
#df_wins_grouped['det_wins_%'] = np.round(100 * df_wins_grouped['det_wins'] / df_wins_grouped['#scenarios'], 2)
df_wins_grouped = df_wins_grouped.merge(df_num_scenarios_per_instance, left_on=['InstanceName', 'Model_target', 'GammaPerc_target'],
                                        right_on=['InstanceName', 'Model', 'GammaPerc'])
df_wins_grouped.head()

In [ ]:
g = sns.catplot(x="ModelName", y="rob_wins_%",
                 col="InstanceName", hue="RTCS_Policy",
                 data=df_wins_grouped.reset_index(), kind="bar", ci=None, orient='v')
g.set_xticklabels(rotation=90)

### Calculating VaR and CVaR

Reference: https://quantdare.com/value-at-risk-or-expected-shortfall/

In [ ]:
df_scenario_cost = df.groupby(by=['InstanceName', 'Model', 'ModelName', 'GammaPerc', 'RTCS_Policy', 'ScenarioId'])\
    .agg({'cost' : sum})
df_scenario_cost.head()

In [ ]:
print('Cost Avg...')
pd.pivot_table(df_scenario_cost.reset_index(), values='cost', index=['InstanceName', 'RTCS_Policy'],
                    columns=['ModelName'], aggfunc=np.mean).reindex(['deterministic-0','deterministic-50','deterministic-100','robust-budget-0', 'robust-budget-20','robust-budget-40',
                                                                                    'robust-budget-60', 'robust-budget-80', 'robust-budget-100', 'robust-box'], axis=1)

In [ ]:
print('VaR: 95% certain that losses will not exceed the value of...')
pd.pivot_table(df_scenario_cost.reset_index(), values='cost', index=['InstanceName', 'RTCS_Policy'],
                    columns=['ModelName'], aggfunc=value_at_risk).reindex(['deterministic-0','deterministic-50','deterministic-100',
                                                                           'robust-budget-0', 'robust-budget-20','robust-budget-40',
                                                                                    'robust-budget-60', 'robust-budget-80', 'robust-budget-100', 'robust-box'], axis=1)

In [ ]:
print('CVaR: estimate of expected losses sustained in the worst 1 - x% of scenarios')
df_cvar = pd.pivot_table(df_scenario_cost.reset_index(), values='cost', index=['InstanceName', 'RTCS_Policy'],
                    columns=['ModelName'], aggfunc=expected_shortfall).reindex(['deterministic-0','deterministic-50','deterministic-100',
                                                                                'robust-budget-0', 'robust-budget-20','robust-budget-40',
                                                                                    'robust-budget-60', 'robust-budget-80', 'robust-budget-100', 'robust-box'], axis=1)
df_cvar
#df_cvar.style.format(precision=0, na_rep='MISSING', thousands=" ",
#               formatter={('Decision Tree', 'Tumour'): "{:.2f}",
#                           ('Regression', 'Non-Tumour'): lambda x: "$ {:,.1f}".format(x*-1e6)
#                          })

### TODO: Incluir tabela com valor esperado, SD, VaR 95% e CVaR 95% para cada modelo e estratégia acima.

### Figure. Split violin plot with the costs of each scenario, comparing Rob x Det

In [ ]:
df_box_vs_det = df_scenario [(df_scenario['ModelName'] == 'robust-budget-60') | (df_scenario['ModelName'] == 'deterministic')]
df_box_vs_det_1 = df_box_vs_det[(df_box_vs_det['InstanceName'] == 'A_instance2_1000s_skewed-left')]
df_box_vs_det['ModelName'].unique()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
plot4 = sns.catplot(x="RTCS_Policy", y="cost", hue="ModelName",
            kind="violin", split=True,
            palette="pastel", data=df_box_vs_det_1)
plot4.set_xticklabels(rotation=90)

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = plt.subplots(figsize=a4_dims)
plt.figure(figsize=(20,5))
sns.catplot(x="cost", y="RTCS_Policy", hue="ModelName", row="InstanceName", 
            kind="violin", bw=.15, cut=0, 
            data=df_scenario,
            height=25, # make the plot 15 units high
            aspect=0.5) # height should be 2 times width

In [ ]:
#a4_dims = (11.7, 8.27)
#fig, ax = plt.subplots(figsize=a4_dims)
plt.figure(figsize=(20,5))
sns.catplot(x="cost", y="RTCS_Policy", hue="ModelName", 
            kind="violin", bw=.15, cut=0, 
            data=df_scenario[(df_scenario['InstanceName'] == 'spring')],
            height=25, # make the plot 15 units high
            aspect=0.5) # height should be 2 times width

### TODO Fazer um kde distribution plot dos custos do RTCS obtidos nas simulacoes: robusto-gamma vs. deterministico

### TODO Fazer uma tabela com as medidas estatisticas (para cada distribuicao usada) de cada simulacao, incluindo valor esperado, SD, percentis 95, 99 e valor maximo observado empiricamente.

In [ ]:
df.groupby(by=['InstanceName', 'ModelName', 'RTCS_Policy', 'GammaPerc', 'ScenarioId', 't']).count()

In [ ]:
for experiment_folder in experiment_folder_list:
    for instance_group in instance_group_list:
        instance_list = get_instance_list(project_folder, antoine_instances_folder, toy_instances_folder, instance_group)
        print(instance_group, instance_list)
        for model in simulated_model_list:
            for forecast_type in forecast_type_list:
                print(model, forecast_type)